Datasets:
Complete Binary, w/ background - sizes 1 and 10 and highly mixed 5
Complete Binary, no background - sizes 1 and 10 and incremental 5
Custom Smoothing, w/ background - sizes 1 and 10 - This is the baseline
Custom Smoothing, no background - sizes 1 and 10


Loss settings:
Custom Smoothing, enabled - Loss A -This is the baseline
Custom Smoothing, disabled - Loss A
No Smoothing, disabled - Loss A or B 
Proper Smoothing, disabled - Loss B


Model Types: 
Standard Dilation, No Attention - This is the baseline
Standard Dilation, Attention
Reduced Dilation, No Attention
Reduced Dilation, Attention

Todo: Need to make a reduced dilation version of the model - done, model takes dilation scaler
Confirm that if no smoothing and early reward disabled, Loss A and Loss B are equivalent - confirmed, so long as parameters and thresholds are the same

Checklist of datasets:

 Complete Binary, w/ background

 Size 1 - COMPLETE

 Size 10

 Highly mixed 5


 Complete Binary, no background

 Size 1

 Size 10

 Incremental 5 - have a custom smoothed version w/ background that can be converted

 Custom Smoothing, w/ background (This is the baseline)

 Size 1 - COMPLETE

 Size 10 - COMPLETE

 Custom Smoothing, no background

 Size 1

 Size 10


In [5]:
import time
import sys
import os
import glob
import math
import threading
import concurrent.futures as cf
import random
import re

import numpy as np
import pandas as pd
import tensorflow as tf
from keras import Input, Model, layers, metrics, losses, callbacks, optimizers, models, utils
from keras import backend as K
import gc
import keras_tuner as kt
from pyfaidx import Fasta

import sys
import os

datasets_path = "../../Datasets/"
models_path = "../../Models/"

In [20]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
import IEModules as iem

2025-04-08 00:31:57.361050: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-08 00:31:57.452228: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-08 00:31:57.478923: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-08 00:31:57.644693: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-08 00:31:59.193106: W tensorflow/compiler/tf2

In [6]:
iem.Genetic_Data_Pipeline.test_function("Howdy!")

You entered Howdy! into test_function


'Howdy!'

In [8]:
import inspect
functions = inspect.getmembers(iem.Genetic_Data_Pipeline, inspect.isfunction)
for name, func in functions:
    print(name)

Input
_bytes_feature
_float_feature
_int64_feature
build_chunk_data_for_indices
build_dataset_from_tfrecords
bytes_feature
calculate_introns
compute_chunk_indices
convert_and_write_tfrecord
convert_labels_to_binary
float_feature_list
int_feature_list
label_sequence_local
load_gtf_annotations
main
one_hot_encode_reference
pad_encoded_seq
pad_labels
parse_chunk_example
search_gtf_by_range
serialize_chunk_example
serialize_example_with_metadata_no_convert
split_tfrecords
stream_shuffled_records
swap_columns_if_needed
test_dataset_from_tfrecords
test_function
trim_chr_genome
tvt_split_tfrecords
write_shuffled_records_to_single_tfrecord
write_tfrecord_in_shards_hybrid
write_to_shard_with_threads


Check Genetic_Data_Pipeline for correctness

Check Custom_Models for correctness

Need to make a shard shuffler module probably that goes from output of genetic_data_pipeline to fully shuffled test-train-split

In [3]:
iem.Genetic_Data_Pipeline.dataset_pipeline_help()


          1. build_initial_shards function generates 4 tfrecord shards at each shift fed to it.
          I recommend giving a list of 1 window shift.  Currently paths are hardcoded.  
          The first 4*n characters of the shards will be the shift numbers where n is 
          the number of shifts fed to the function.
          
          2. split_tfrecords takes an input and output directory and number of file splits
          and splits each TFRecord in the input_directory (assumed to be gzipped TFRecords)
          into 'num_splits' smaller TFRecords.
          
          3. write_shuffled_records_to_single_tfrecord builds a single big, shuffled, 
          gzip-compressed TFRecord file 
          
          Args:
            input_dir (str): Directory containing the source TFRecord files.
            allowed_indices (list): List of allowed starting indices.
            output_filepath (str): Full path to the output TFRecord file.
            
          4. tvt_split_records tes

Checklist of datasets:

 Complete Binary, w/ background

 Size 1 - COMPLETE

 Size 10 - COMPLETE

 Highly mixed 5                                 Need to build highly mixed 5 and convert to binary


 Complete Binary, no background

 Size 1 - background removed when loading into model (Data_Functions.py)

 Size 10 - background removed when loading into model (Data_Functions.py)

 Incremental 5 - have a custom smoothed version w/ background that can be converted   conversion ongoing

 Custom Smoothing, w/ background (This is the baseline)

 Size 1 - COMPLETE

 Size 10 - COMPLETE

 Custom Smoothing, no background

 Size 1 - background removed when loading into model (Data_Functions.py)

 Size 10 - background removed when loading into model (Data_Functions.py)



 Size 10 has 2365261 records

In [6]:
iem.Genetic_Data_Pipeline.convert_and_write_tfrecord(datasets_path+"AugDataSets/all_ten_shuffled.tfrecord.gz", datasets_path+"AugDataSets/binary_ten_shuffled.tfrecord.gz")

I0000 00:00:1744095490.509284 1330704 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744095490.699680 1330704 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744095490.699799 1330704 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744095490.704425 1330704 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744095490.704488 1330704 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [10]:
iem.Genetic_Data_Pipeline.convert_and_write_tfrecord(datasets_path+"AugDataSets/fifths_incremental_shuffled.tfrecord.gz", datasets_path+"AugDataSets/binary_fifths_incrememntal.tfrecord.gz")

In [ ]:
iem.Genetic_Data_Pipeline.transform_tfdataset(datasets_path+"AugDataSets/AllTen", datasets_path+"AugDataSets", 45, 0.5)
print("Done")